# snowflakeR Quickstart -- Local Environment

This notebook is for **local R environments** (RStudio, Posit Workbench, JupyterLab with R kernel).
For Snowflake Workspace Notebooks, use `workspace_quickstart.ipynb`.

**Before you start:** Copy `notebook_config.yaml.template` to `notebook_config.yaml`
and edit it with your account, warehouse, database, and schema.

**Sections:**
1. Setup
2. Connect & set execution context
3. Queries & Table Operations
4. DBI & dbplyr via RSnowflake (optional)
5. Visualization with ggplot2
6. Cleanup

## 1. Setup

Install snowflakeR and set up the Python environment (one time only).

```r
# install.packages("pak")
# pak::pak("Snowflake-Labs/snowflakeR")
# snowflakeR::sfr_install_python_deps()
```

In [ ]:
library(snowflakeR)

---
## 2. Connect & Set Execution Context

`sfr_load_notebook_config()` reads `notebook_config.yaml` and runs
`USE WAREHOUSE / DATABASE / SCHEMA` to set the execution context.

All table references use fully qualified names via `sfr_fqn()` for
consistency with Workspace Notebooks.

In [ ]:
# Connect (reads ~/.snowflake/connections.toml or config connection params)
conn <- sfr_connect()

# Load config and set execution context
conn <- sfr_load_notebook_config(conn)
conn

---
## 3. Queries & Table Operations

In [ ]:
# Run a SQL query
result <- sfr_query(conn, "SELECT CURRENT_TIMESTAMP() AS now, CURRENT_USER() AS user_name")
result

In [ ]:
# Write a data.frame to Snowflake (fully qualified name)
sfr_write_table(conn, sfr_fqn(conn, "SFR_MTCARS"), mtcars, overwrite = TRUE)

In [ ]:
# List tables
tables <- sfr_list_tables(conn)
head(tables, 10)

In [ ]:
# Read it back (fully qualified name)
df <- sfr_read_table(conn, sfr_fqn(conn, "SFR_MTCARS"))
head(df, 5)

In [ ]:
# Describe columns
sfr_list_fields(conn, sfr_fqn(conn, "SFR_MTCARS"))

---
## 4. DBI & dbplyr via RSnowflake (optional)

For standard DBI-compliant database access and dbplyr integration, use the
`RSnowflake` package. You can obtain an `RSnowflake` connection from your
`sfr_connection` via `sfr_dbi_connection()`, or create one directly with
`DBI::dbConnect(RSnowflake::Snowflake(), ...)`.

In [ ]:
if (requireNamespace("RSnowflake", quietly = TRUE)) {
  library(RSnowflake)
  library(DBI)

  # Get an RSnowflake DBI connection from the sfr_connection
  dbi_con <- sfr_dbi_connection(conn)

  DBI::dbGetQuery(dbi_con, "SELECT 42 AS answer")
  DBI::dbExistsTable(dbi_con, sfr_fqn(conn, "SFR_MTCARS"))
} else {
  message("RSnowflake not installed -- skipping DBI section.")
  message("Install with: install.packages('path/to/RSnowflake', repos = NULL, type = 'source')")
}

In [ ]:
if (exists("dbi_con")) {
  library(dplyr)
  library(dbplyr)

  # Lazy reference to Snowflake table via the RSnowflake DBI connection
  cars_tbl <- tbl(dbi_con, I(sfr_fqn(conn, "SFR_MTCARS")))

  # dplyr pipeline -- generates SQL, runs on collect()
  # Column names are uppercase (Snowflake convention)
  summary <- cars_tbl |>
    group_by(CYL) |>
    summarise(
      n       = n(),
      avg_mpg = mean(MPG, na.rm = TRUE),
      avg_hp  = mean(HP, na.rm = TRUE)
    ) |>
    arrange(CYL) |>
    collect()

  summary
} else {
  message("Skipping dbplyr section (RSnowflake not available).")
}

---
## 5. Visualization with ggplot2

In [ ]:
library(ggplot2)

df <- sfr_read_table(conn, sfr_fqn(conn, "SFR_MTCARS"))

ggplot(df, aes(x = wt, y = mpg, color = factor(cyl))) +
  geom_point(size = 3) +
  labs(title = "MPG vs Weight by Cylinder Count",
       x = "Weight (1000 lbs)", y = "Miles per Gallon",
       color = "Cylinders") +
  theme_minimal()

---
## 6. Cleanup

In [ ]:
# Uncomment to clean up demo objects
# (commented out to avoid accidental deletion on Run All)
#
# sfr_execute(conn, paste("DROP TABLE IF EXISTS", sfr_fqn(conn, "SFR_MTCARS")))
# cat("Cleanup complete.\n")